<a href="https://colab.research.google.com/github/natsakh/IAD/blob/main/Pr_5/5_4_LogRegr_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random

torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


<small>[https://docs.pytorch.org/vision/0.8/datasets.html](https://docs.pytorch.org/vision/0.8/datasets.html)</small>

<small>[https://docs.pytorch.org/vision/stable/_modules/torchvision/datasets/mnist.html](https://docs.pytorch.org/vision/stable/_modules/torchvision/datasets/mnist.html)</small>

<small>[https://docs.pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html)</small>


In [2]:
 # MNIST dataset (images 28x28 → tensors in [0,1])
transform = transforms.ToTensor()

train_set = datasets.MNIST(root="./data", train=True,  download=True, transform=transform)
test_set  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
#train_set, test_set —  об’єкти класу torchvision.datasets.MNIST

100%|██████████| 9.91M/9.91M [00:00<00:00, 56.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.65MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.08MB/s]


In [3]:
print(type(train_set))
print(len(train_set))
print(len(test_set))

<class 'torchvision.datasets.mnist.MNIST'>
60000
10000


In [4]:
img, label = train_set[0]

print("Type:", type(img))
print("Shape:", img.shape) #(1 канал, тобто чорно-біле зображення 28×28 пікселів)
print("Label:", label)
print("Min:", img.min().item(), "Max:", img.max().item())

Type: <class 'torch.Tensor'>
Shape: torch.Size([1, 28, 28])
Label: 5
Min: 0.0 Max: 1.0


In [5]:
# DataLoaders
batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)

In [6]:
# Model: Logistic Regression = Linear(784 -> 10)
class LogisticRegressionMNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28*28, 10)  # logits for 10 classes

    def forward(self, x):
        x = x.reshape(x.size(0), -1)           # flatten: [B, 1, 28, 28] -> [B, 784]
        logits = self.linear(x)             # no softmax here!
        return logits  #logit — це лінійний вихід моделі до застосування сигмоїди (або softmax)

model = LogisticRegressionMNIST().to(device)
print(model)

LogisticRegressionMNIST(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [7]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable params:", num_params)   # 784*10 + 10 = 7850


Trainable params: 7850


<small>[https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)</small>

In [8]:
# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [9]:
# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)          # [B,10]
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # test accuracy each epoch
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = correct / total

    print(f"[{epoch+1}/{epochs}] loss={avg_loss:.4f} | test acc={acc*100:.2f}%")


[1/5] loss=0.6578 | test acc=90.27%
[2/5] loss=0.3587 | test acc=91.24%
[3/5] loss=0.3177 | test acc=91.90%
[4/5] loss=0.2985 | test acc=92.02%
[5/5] loss=0.2869 | test acc=92.26%
